In [5]:
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision 

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device Using: {device}")

Device Using: cpu


In [7]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

random_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

])
training_dataset = torchvision.datasets.CIFAR10(root="../data",download=True,train=True,transform=random_transforms)
train_loader = DataLoader(training_dataset,num_workers=2,shuffle=True,batch_size=128)


test_dataset = torchvision.datasets.CIFAR10(root="../data",train=False,download=True,transform=random_transforms)
test_loader = DataLoader(test_dataset,shuffle=True,num_workers=2,batch_size=128)

100.0%


In [9]:
print(f"Len of our training Samples {len(training_dataset)}")
print(f"Len of our test Samples {len(test_dataset)}")

name = "Zaid Ghaffar"

Len of our training Samples 50000
Len of our test Samples 10000


In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.Conv2d(64, 64, 3, padding=1),
            
            # Pooling and dropout
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),
            nn.Dropout(0.5),
            
            # Fully connected layers
            nn.Linear(64 * 4 * 4, 512),
            nn.Linear(512, 10),
            
            # Activation
            nn.ReLU()
        )

    def forward(self,x):
        return self.model(x)


In [16]:
from sklearn.metrics import accuracy_score,precision_score
from datetime import datetime
import os
from torch.utils.tensorboard import SummaryWriter

In [ ]:
class Trainer:
    def __init__(self,train_loader,test_loader,Saved_model="CIFAR10_CNN"):
        self.model = Model()
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.loss_fun =nn.CrossEntropyLoss()
        self.optimizer = Adam(self.model.Paramerts(),lr=0.001)

        # Seeting up TensorBoard
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        PATH_LOG_DIR = r""
        self.log_dir = os.path.join(PATH_LOG_DIR,f"{Saved_model}_{timestamp}")
        self.writer = SummaryWriter(self.log_dir)

    def train_model(self,epochs):
        self.model.train()
        training_metrics = {"train_Loss": [],"train_accuracy":[]}
        for epoch in epochs:
            running_loss = {"Loss":0.00,"total":0,"correct":0}
            for i,(inputs,labels) in enumerate(self.train_loader):
                inputs,labels = inputs.to(device),labels.to(device)
                self.optimizer.zero_grad()
                yhat = self.model(input)
                loss = self.loss_fun(yhat,labels)
                loss.backward()
                self.optimizer.step()
                running_loss["Loss"] = loss.item()
                _,predicted =torch.max(yhat,1)
                running_loss["total"] = labels.size(0)
                running_loss["correct"] = (predicted == labels).sum().item()

                print(f"Epochs:        {epoch+1}/{epochs} \n   Loss Per Epoc: {running_loss["Loss"]}  \n Accuracy: {running_loss["correct"]*100/running_loss["total"]}")
            epoch_loss = running_loss["Loss"]/len(self.train_loader)
            epoch_accuracy = running_loss["correct"] *100/running_loss["total"]
            training_metrics["train_Loss"].append(epoch_loss)
            training_metrics["train_accuracy"].append(epoch_accuracy)

            self.writer.add_scaler(f"Loss/Epoch:    {training_metrics["train_Loss"],epoch}")
            self.writer.add_scaler(f"Accuracy/Epoch:    {training_metrics["train_accuracy"],epoch}")

    def Test_model(self):
        pass

